In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
import emoji
import re

nltk.download('punkt')
nltk.download('punkt_tab')

def clean_and_extract_sentences(url, keywords):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Fehler beim Abrufen von {url}: {e}")
        return []

    # parse HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    for script in soup(["script", "style", "nav", "footer", "header", "noscript"]):
        script.extract()

    text = soup.get_text(separator=' ')

    text = re.sub(r'\s+', ' ', text).strip()

    sentences = nltk.sent_tokenize(text, language='german')

    found_data = []

    for sentence in sentences:
        sentence_clean = sentence.strip()

        # --- FILTER ---

        # 1. Word Count (between 5 and 40 words)
        word_count = len(sentence_clean.split())
        if not (5 <= word_count <= 40):
            continue

        # 2. No Emojis
        if emoji.emoji_count(sentence_clean) > 0:
            continue

        # 3. Keyword-Check (case-insensitive)
        sentence_lower = sentence_clean.lower()

        keyword_found = False
        for k in keywords:
            if k in sentence_lower:
                keyword_found = True
                break

        if keyword_found:
            found_data.append({
                'Satz': sentence_clean,
                'URL': url
            })

    return found_data

# --- KONFIGURATION ---

keywords_raw = """
a+++, a++, a+, abbaubar, abhängig, abfall, abhaengig, aise, aktiv, alternativ, anbau, anerkannt, arm, artenvielfalt, asc, aus der nähe, aus der naehe, ausgleich, ausgeglichen, ausstoß, ausstoss, aufbereit, baum, baeume, bäume, b-corp, bci, behandelt, belassen, besser, better cotton initiative, bio, blue sign, boden, bewusst, carbon, chemie, chemikalie, chemisch, clean, climatepartner, cmia, co2, kompens, conscious, cradle to cradle, demeter, dueng, düng, earth, eco, effizient, effizienz, emission, emitt, enkel, engagement, engagiert, energie, engel, erde, erneuerbar, ethisch, ethical, fair, fckw, fla, fsc, fußabdr, frei, freundlich, fwf, ganzheitlich, gerecht, gepa, gentechni, gespritzt, gift, global organic textile standard, gots, green, gruen, grün, grs, guetezeichen, gütezeichen, gut, hand, haltbar, haltungsform, heimat, hybrid, hydro, igep, impact, einfluss, ivn, klima, kohlendioxid, kohlenstoffdioxid, kompens, konform, kontroll, koralle, kreislauf, kunststoff, label, lachgas, landbau, langlebig, lebensdauer, lebenszyklus, local, lokal, luft, methan, mikroplastik, msc, müll, muell, nachhaltig, nachgewiesen, nachwachsend, natur, natür, natuer, net zero, net-zero, netto-null, netto null, nettonull, ncp, negativ, neutral, nordischer schwan, ocean, oeco, oeko, oekotex, oeko-tex, öko, ökotex, öuz, ok power, organic, ozean, offset, papier, pflanz, planet, plastik, positiv, pvc, pur, rainforest alliance, recycel, recycl, reduz, reduziert, region, rein, repar, ressourcen, risiko, rohstoff, rueckgew, rückgew, sa8000, sauber, schadstoff, schädlich, schaedlich, schon, schmutz, schutz, schuetz, schütz, sdg, self made, selfmade, selbstgemacht, sicher, siegel, solar, sonne, spar, standard, strom, sustainable development goals, sustainab, tier, tierleid, toxisch, tco certified, transport, transformation, transparent, treibhausgas, tüv, umwelt, ursprung, urkunde, verantwort, verpflicht, verarbeit, verpack, vertraeglich, verträglich, vegan, vegetari, von hand, von hier, wald, wasser, wiederverwend, wind, wohlergehen, wwf, wrap, xertifix, zertifi, zeugnis, ziel, zukunft
"""

keyword_list = [k.strip().lower() for k in keywords_raw.split(',')]


input_csv = 'data/urls.csv'

df_urls = pd.read_csv(input_csv, sep=';')

urls_to_analyze = df_urls['Seite (URL)'].dropna().unique().tolist()

urls_to_analyze = [u for u in urls_to_analyze if str(u).startswith('http')]


# --- PROGRAM ---

all_results = []

print(f"Starte Analyse von {len(urls_to_analyze)} URLs mit {len(keyword_list)} Keywords...")

for url in urls_to_analyze:
    print(f"Verarbeite: {url}")
    results = clean_and_extract_sentences(url, keyword_list)
    all_results.extend(results)

df = pd.DataFrame(all_results)

# Export
output_filename = "analyse_ergebnisse.xlsx"
df.to_excel(output_filename, index=False)

print(f"Fertig! {len(df)} Sätze gefunden. Gespeichert in '{output_filename}'.")